In [2]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']


In [16]:
sentences = open("train.txt.target.txt").readlines()
sentences[:10]

['to whom did the virgin mary allegedly appear in 1858 in lourdes france ?\n',
 'what is in front of the notre dame main building ?\n',
 'the basilica of the sacred heart at notre dame is beside to which structure ?\n',
 'what is the grotto at notre dame ?\n',
 'what sits on top of the main building at notre dame ?\n',
 'when did the scholastic magazine of notre dame begin publishing ?\n',
 "how often is notre dame 's the juggler published ?\n",
 'what is the daily student paper at notre dame called ?\n',
 'how many student news papers are found at notre dame ?\n',
 'in what year did the student paper common sense begin publication at notre dame ?\n']

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = open("train.txt.target.txt").readlines()

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

# #Print the embeddings
# for sentence, embedding in zip(sentences, embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

In [8]:
import numpy as np

In [12]:
np.save("NQG.train.txt.target.sembeddings.all-MiniLM-L6-v2",embeddings)

In [14]:
embeddings

array([[-0.01816939,  0.05504933, -0.00028672, ..., -0.02605437,
         0.11045023, -0.05906297],
       [ 0.03145022,  0.07019895,  0.04281677, ..., -0.04420455,
         0.02051509,  0.0663435 ],
       [ 0.0515607 ,  0.06941284,  0.01818386, ...,  0.0027039 ,
         0.09355298, -0.00370873],
       ...,
       [ 0.01568051,  0.03858426, -0.04172188, ...,  0.02713734,
         0.01242578, -0.02956982],
       [-0.029038  ,  0.05255385, -0.01565575, ..., -0.03874199,
        -0.01462394, -0.0176759 ],
       [-0.07209045, -0.05081196,  0.00075677, ...,  0.06404006,
         0.04356275,  0.06448354]], dtype=float32)

In [6]:
from gpt4all import GPT4All

In [8]:
a = GPT4All("nous-hermes-llama2-13b.Q4_0.gguf")

100%|█████████████████████████████████████████████████████████████████████████████| 7.37G/7.37G [22:38<00:00, 5.42MiB/s]


In [14]:
a.generate("What is a test?")

'\nA test is an assessment intended to measure the knowledge, skill or ability of a person or group. Tests can be used for various purposes such as evaluation, selection and certification. They are commonly administered in educational institutions, workplaces and other organizations to determine whether individuals meet certain standards or criteria.\nWhat types of tests are there?\nThere are several different types of tests that serve a variety of purposes:\n- Achievement Tests: These measure what students have learned after completing an instructional program or course. They can be used to assess the effectiveness of teaching methods and materials, as well as individual student progress.\n- Aptitude Tests: These aim to predict future performance in a certain area based on current abilities. For example, an employer might use an aptitude test to determine if a job candidate has the necessary skills for a particular position.\n- Diagnostic Tests: These are used to identify specific are

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "TheBloke/Mistral-7B-v0.1-AWQ"

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", trust_remote_code=False)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Fusing layers...: 100%|█████████████████████████████████████████████████████████████████| 32/32 [00:07<00:00,  4.04it/s]


In [8]:
import tqdm
import json

sentences = open("train.txt.target.txt").readlines()
sentences_ratings = []

prompt_template='''"Bloom's Taxonomy is a hierarchical ordering of cognitive skills that can help teachers and students in the classroom. There are multiple levels of understanding: 
1 - Knowledge
2- Comprehension
3 - Application
4 - Analysis
5 - Synthesis
6 - Evaluation.
For example, the question '{}' is considered to be of level '''

idx = 0
for prompt in tqdm.notebook.tqdm(sentences):
    tokenizer.pad_token = tokenizer.eos_token
    tokens = tokenizer(
        prompt_template.format(prompt.strip()),
        return_tensors='pt',
        padding=True
    ).input_ids.cuda()

    # Generate output
    generation_output = model.generate(
        tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=10,
        max_new_tokens=1
    )
    sentences_ratings.append(tokenizer.decode(generation_output[0][-1]))
    
    if idx % 100 == 0:
        print("Output: ", tokenizer.decode(generation_output[0][-1]))
        with open("NQG.train.txt.target.llmratings.all-MiniLM-L6-v2.json", "w") as f:
            json.dump(sentences_ratings, f)
    idx +=1

  0%|          | 0/86635 [00:00<?, ?it/s]

Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  2
Output:  1
Output:  1
Output:  1


KeyboardInterrupt: 

In [14]:
all_labels = [-1 for _ in sentences]
for idx, label in enumerate(sentences_ratings):
    try: 
        label = int(label)
    except:
        label = -2
    all_labels[idx] = label

In [15]:
json.dump(all_labels, open("NQG.train.txt.target.llmratingsfull.all-MiniLM-L6-v2.json", "w"))

In [16]:
import pandas as pd

In [23]:
pd.DataFrame({"sentences": sentences, "label":all_labels}).head(49)

,sentences,label
0,to whom did the virgin mary allegedly appear i...,1
1,what is in front of the notre dame main buildi...,1
2,the basilica of the sacred heart at notre dame...,1
3,what is the grotto at notre dame ?\n,1
4,what sits on top of the main building at notre...,1
5,when did the scholastic magazine of notre dame...,1
6,how often is notre dame 's the juggler publish...,1
7,what is the daily student paper at notre dame ...,1
8,how many student news papers are found at notr...,1
9,in what year did the student paper common sens...,1


In [56]:
df = pd.DataFrame({"sentences": sentences, "label":all_labels})
print(*[(el[1][0], el[1][1]) for el in df[df["label"] > 1].iterrows()], sep="\n\n")

('what was the review of politics inspired by ?\n', 2)

('what threat does the global adaptation index study ?\n', 3)

('what type of education was pushed at notre dame before its embracing of national standards ?\n', 2)

('those who attended a jesuit college may have been forbidden from joining which law school due to the curricula at the jesuit institution ?\n', 5)

("for whos glory did father o'hara believed that the notre dame football team played ?\n", 2)

('in the time that hesburgh was president of notre dame by what factor did the operating budget increase ?\n', 3)

('what caused many intellectual catholics to leave europe in the 1930s ?\n', 2)

('where did the fighting irish hockey team compete prior to a move to hockey east , in terms of conference ?\n', 2)

('what university is notre dame tied with in terms of most heisman trophy winners ?\n', 2)

("against which opponent did knute rockne tell his team to `` win one for the gipper ? ''\n", 2)

("which person portrayed knute 

In [57]:
df = pd.DataFrame({"sentences": sentences, "label":all_labels})
print(*[(el[1][0], el[1][1]) for el in df[df["label"] > 4].iterrows()], sep="\n\n")

('those who attended a jesuit college may have been forbidden from joining which law school due to the curricula at the jesuit institution ?\n', 5)

('which prominent star felt the 2009 female video of the year award should have went to beyoncé instead of taylor swift ?\n', 6)

('hoe did everyone learn that beyonce performed for kaddafi ?\n', 5)

("who inspires beyoncé because `` she does it all ? ''\n", 6)

("in what year was the slang term from a title of a destiny 's child song that is also used to describe beyoncé put in the dictionary ?\n", 6)

('what about beyonce has influenced many entertainers ?\n', 5)

("which national event caused beyonce to produce `` demand a plan ? ''\n", 6)


In [58]:
import tqdm
import json

sentences = open("train.txt.target.txt").readlines()
sentences_ratings = []

prompt_template='''"Bloom's Taxonomy is a hierarchical ordering of cognitive skills that can help teachers and students in the classroom. There are multiple levels of understanding: 
1 - Knowledge, which relates to identification and recall of information,
2- Comprehension, which relates to organization and selection of facts and ideas,
3 - Application, which relates to the use of facts, rules and principles,
4 - Analysis, which relates to separation of a whole into component parts,
5 - Synthesis, which relates to the combination fo ideas to form a new whole,
6 - Evaluation. which relates to developing opinions, judgements or decisions.

For example, the question '{}' is considered to be of level '''

idx = 0
for prompt in tqdm.notebook.tqdm(sentences):
    tokenizer.pad_token = tokenizer.eos_token
    tokens = tokenizer(
        prompt_template.format(prompt.strip()),
        return_tensors='pt',
        padding=True
    ).input_ids.cuda()

    # Generate output
    generation_output = model.generate(
        tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=10,
        max_new_tokens=1
    )
    sentences_ratings.append(tokenizer.decode(generation_output[0][-1]))
    
    if idx % 100 == 0:
        print("Output: ", tokenizer.decode(generation_output[0][-1]))
        with open("NQG.train.txt.target.llmratings-v2.all-MiniLM-L6-v2.json", "w") as f:
            json.dump(sentences_ratings, f)
    idx +=1

  0%|          | 0/86635 [00:00<?, ?it/s]

Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  1
Output:  2
Output:  1
Output:  2
Output:  2


KeyboardInterrupt: 

In [59]:
all_labels2 = [-1 for _ in sentences]
for idx, label in enumerate(sentences_ratings):
    try: 
        label = int(label)
    except:
        label = -2
    all_labels2[idx] = label

In [60]:
df = pd.DataFrame({"sentences": sentences, "label":all_labels, "label2":all_labels2})


In [64]:
print(*[(el[1][0], el[1][1]) for el in df[df["label"] > 1].iterrows()], sep="\n\n")

('what was the review of politics inspired by ?\n', 2)

('what threat does the global adaptation index study ?\n', 3)

('what type of education was pushed at notre dame before its embracing of national standards ?\n', 2)

('those who attended a jesuit college may have been forbidden from joining which law school due to the curricula at the jesuit institution ?\n', 5)

("for whos glory did father o'hara believed that the notre dame football team played ?\n", 2)

('in the time that hesburgh was president of notre dame by what factor did the operating budget increase ?\n', 3)

('what caused many intellectual catholics to leave europe in the 1930s ?\n', 2)

('where did the fighting irish hockey team compete prior to a move to hockey east , in terms of conference ?\n', 2)

('what university is notre dame tied with in terms of most heisman trophy winners ?\n', 2)

("against which opponent did knute rockne tell his team to `` win one for the gipper ? ''\n", 2)

("which person portrayed knute 

In [67]:
print(*[(el[1][0], el[1][2]) for el in df[df["label2"] > 4].iterrows()], sep="\n\n")

('from where did anton-hermann chroust come to reach notre dame ?\n', 5)

("beyonce has said that who embodies the `` definition of inspiration and a strong woman '' ?\n", 5)

("vh1 declared what song the `` greatest song of the 2000s ? ''\n", 5)


In [63]:
df[df["label"] != df["label2"]].head(49)

,sentences,label,label2
17,before the creation of the college of engineer...,1,2
64,what was the review of politics inspired by ?\n,2,1
71,what threat does the global adaptation index s...,3,1
73,what percentage of students at notre dame are ...,1,3
75,for what cause is money raised at the bengal b...,1,2
83,what percentage of notre dame students feel th...,1,2
96,those who attended a jesuit college may have b...,5,3
102,"catholic people identified with notre dame , w...",1,2
105,where did notre dame students and the kkk have...,1,2
111,around the time that rev. cavanaugh became pre...,1,2
